- 对python数据类型和C数据的转换、包装

- C语言中数据类型是如何在python中转换的

- python如何使用C的方法、函数（如果传入参数为python数据类型则需要类型转换，如果传入从ctypes定义的C的数据类型则不需要转换）

# 加载 dll 和取出函数

```python
from ctypes import *
dll = cdll.LoadLibrary(dllpath)   #dllpath是字符串
dll = windll.LoadLibrary(dllpath)
```

上面两行使用哪一行,取决于导出函数的调用规范(cdecl 或 stdcall).也可以使用下面两行代替:

```python
dll = CDLL(dllpath)    #注意和上面大小写的区别
dll = WinDLL(dllpath)
```

注意,这里使用的 dll 必须和 python 平台匹配,比如都是 32 位的或者都是 64 位的。因为本质上是一个 exe 加载一个 dll，无法跨平台。

加载 dll 后,可直接得到 dll 中的导出函数地址.

```python
func = dll.func_name  #func_name 是dll的导出函数
```

有时动态链接库导出 c++函数时，并不是有效的 Python 标识符，例如 "??2@YAPAXI@Z" 。这种情况下，必须使用 getattr 获取函数:

```python
func = getattr(cdll.msvcrt, "??2@YAPAXI@Z")
```

在 Windows 上，有些动态链接库导出函数不是用名字，而是用序号(ordinal)。这时需通过索引获取：

```python
func = cdll.kernel32[1]
```

# 数据类型
上面只是得到了函数地址,还无法进行函数调用.要进行正确的函数调用,需设置好参数和返回值类型.
ctypes 支持的原生数据类型如下:

| ctypes 类型          | C 类型                                         | Python 类型                                |
| -------------------- | ---------------------------------------------- | ------------------------------------------ |
| c_char               | char                                           | 1-character string                         |
| c_wchar              | wchar_t                                        | 1-character unicode string                 |
| c_byte               | char                                           | int/long                                   |
| c_ubyte              | unsigned char                                  | int/long                                   |
| c_bool               | bool                                           | bool                                       |
| c_short              | short                                          | int/long                                   |
| c_ushort             | unsigned short                                 | int/long                                   |
| c_int                | int                                            | int/long                                   |
| c_uint               | unsigned int                                   | int/long                                   |
| c_long               | long                                           | int/long                                   |
| c_ulong              | unsigned long                                  | int/long                                   |
| c_longlong           | \_\_int64 or longlong                          | int/long                                   |
| c_ulonglong          | unsigned \_\_int64 or unsigned long long       | int/long                                   |
| c_float              | float                                          | float                                      |
| c_double             | double                                         | float                                      |
| c_longdouble         | long double float                              | float                                      |
| c_char_p             | char \*                                        | string or None                             |
| c_wchar_p            | wchar_t \*                                     | unicode or None                            |
| c_void_p             | void \*                                        | int/long or None                           |


# 参数和返回值

**设置函数的参数类型**
使用函数的 argtypes 属性,直接赋值为一个 ctypes 类型的列表或元组。设置函数的返回值类型使用函数的 restype 属性。下面是示例代码：
python 中，默认函数返回值是 c_int 型，此类型可以不用显示设置函数的 restype 属性，如果是参数类型是 c_int 型则需要设置。

```python
fun.argtypes = (c_int, c_int,c_int, c_void_p) #设置函数参数类型为 int,int,int,void *
fun.restype  = c_float #设置返回值类型为 float
```

None、整数、字节串和（unicode）字符串是可以作为本地 Python 对象直接传递给函数调用的。

- None 是作为 C 的 NULL 指针传递。
- 字节串和字符串作为内存块指针传递(char* 或 wchar_t*)。
- Python 整数作为平台相关的 C 语言 int 类型传递，其值会截断到 C 类型。

除了整数、字节串和字符串以外 Python 类型的参数传递，必须使用 ctypes 类型做包装。

在调用函数时，如果使用了错误的参数数量和调用规范时，ctypes 尝试保护调用。不幸的是该功能仅在 Windows 上有用。它通过检查函数返回栈来实现，所以尽管发生了错误，但是函数还是调用了。
这很容易导致当前使用的整个 Python 环境崩溃，所以必须很小心的使用。

除了上述的基本类型，ctypes 还支持自定义的结构体和联合体，它们可以出现在函数的参数或返回值中。

例如：
```python
func.argtypes = (ctypes.c_int, ctypes.c_int)
func.restype = ctypes.c_uint
handle = func(1001, 1)
```

# 结构体

自定义的结构体和联合体必须继承自 ctypes 的 Structure 和 Union，这两个类都在 ctypes 模块中定义。
每一个子类必须定义"\_fields\_"属性，"\_fields\_"是一个二维的 tuples 列表，
描述类的每个数据成员的字段名和字段类型，这里的字段类型必须是一个 ctypes 类型，
如 c_int，或者任何其他的继承 ctypes 的类型，如 Structure, Union, Array, 指针等。

例如有一个简单结构，包含两个整型 x 和 y，可如下初始化一个结构：

In [1]:
from ctypes import *

In [2]:
class Point(Structure):  
    _fields_ = [('x', c_int),  
                ('y', c_int)]  
p1 = Point(1,2)  
print(p1.x, p1.y) #输出 1 2

1 2


In [3]:
# 类型创建
Point*10

__main__.Point_Array_10

In [4]:
#可以创建复杂的结构体,嵌套了其它结构体。如下：

class RECT(Structure):
    _fields_ = [("upperleft", Point),
                ("lowerright", Point)]

rc = RECT(p1)
print(rc.upperleft.x, rc.upperleft.y)   #输出 1 2
print(rc.lowerright.x, rc.lowerright.y) #输出 0 0

1 2
0 0


In [5]:
#嵌套结构体可以通过下面多种方法初始化：
rc2 = RECT(Point(1,2), Point(3,4))
rc3 = RECT((1,2), (3,4))

In [6]:
# 如结构体用于链表操作，即包含指向结构体指针时
class Test(Structure):  
    pass  

Test._fields_ = [('x', c_int),  
                 ('y', c_char),  
                 ('next', POINTER(Test))]  

# 数组

数组就是序列，包含固定数量的相同类型的实例。推荐的创建数组类型的方式是使用正数和乘号应用到类型:

In [7]:
# 结构体数组
class POINT(Structure):
    _fields_ = [("x", c_int), ("y", c_int)]


#创建数组类型,它是10个Point元素组成的数组。
TenPointsArrayType = POINT * 10
#创建一个数组类的对象。
arr = TenPointsArrayType()
for pt in arr:
    print(pt.x, pt.y)

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0


In [8]:
class MyStruct(Structure):
    _fields_ = [("a", c_int), ("b", c_float),
                ("pts", POINT * 4)]  # 声明类型，相当于C语言的:  POINT pts[4]

print(len(MyStruct().pts))  #输出  4

4


In [9]:
TenIntegers = c_int * 10  # 定义一个int[10]的类型（类）

ii = TenIntegers(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  #显式初始化数组(类)
for i in ii:
    print(i)

1
2
3
4
5
6
7
8
9
10


In [10]:
#高维数组,即数组的数组。
type_int_array_10 = c_int * 10  #先定义一个数组类型
type_int_array_10_10 = type_int_array_10 * 10  #定义数组的数组（即二维数组）
my_array = type_int_array_10_10()  #创建二维数组对象
my_array[1][2] = 3
list(my_array[1])

[0, 0, 3, 0, 0, 0, 0, 0, 0, 0]

# 指针和引用

有时 C 函数需要一个指针指向的数据作为参数，或想向指针指向的内存块写输出数据，或者数据太大不适合传递，这时就需要使用指针或者引用。

ctypes 使用 byref() 函数表示指针概念，该函数也是直接从变量得到指针指向的数据类型。

ctypes 中使用 POINTER 和 pointer 表示指针：

- 在使用 POINTER 时需要设置指向的数据类型

- pointer 则直接从变量中得到一个特定类型的指针。


对内存块的使用，实际上是区分输入/输出的：

- 如果是该内存块是函数的输入（即函数内部是从内存块读数据），则使用指针，即 POINTER() 或 pointer()。

- 如果是该内存块是函数的输出（即函数内部是写数据到内存块），则需使用 create_string_buffer() 函数。

## 指针
**例如创建一个类似于 C 语言的 int \*：**

In [11]:
#创建指针类型,它指向整数
type_p_int = POINTER(c_int)

#定义一个整数,值为4.
v = c_int(4)
#给一个指针变量(p_int)赋值(为变量v的地址).
p_int = type_p_int(v)

print(p_int[0])
print(p_int.contents)  #指针实例有一个contents属性，返回这个指针所指向的对象。

4
c_long(4)


上面这段代码在 C 语言里相当于：

```c
typedef int * type_p_int;
int v = 4;
type_p_int p = &v;
printf("%d",p[0]);
printf("%d",*p);
```

**也可以不经过声明指针类型这一步，直接从变量得到指针，如下：**

In [12]:
#定义一个整数,值为4.
v = c_int(4)
#直接得到v的指针，不需创建指针类型（省去类型声明）。

p_int = pointer(v)
print(p_int[0])
print(p_int.contents)

4
c_long(4)


---

**注意：对指针类型 c_char_p，c_wchar_p，c_void_p 的赋值将会改变其指向的内存区域地址，而不是改变内存块的值(因为 Python 字符串是只读的）。**

**byref()**

ctypes 使用 byref() 函数传递参数引用。
通常使用 byref()的地方同样也可用指针函数 pointer()，但 pointer()作为参数通常会额外创建一个指针对象，
如果并不需要再次使用该指针对象的话，使用 byref() 会更快。

---

**内存块**

各种指针类型（c_char_p，c_wchar_p，c_void_p）指向的内存块实际上都是只读的。
如果某个函数需要一个输入内存块保存输出值，不能传递这些指针。
我们需要一个可写的内存块，使用 create_string_buffer()**开辟内存空间。**

In [13]:
#创建3字节长的buf，且初始化为0
p = create_string_buffer(3)
print(sizeof(p), repr(p.raw))

3 b'\x00\x00\x00'


In [14]:
#创建一个字符串(包括结尾的0)的buf
p = create_string_buffer(b"Hello")
print(sizeof(p), repr(p.raw))

6 b'Hello\x00'


In [15]:
p = create_string_buffer(b"Hello", 10)
print(sizeof(p), repr(p.raw))

10 b'Hello\x00\x00\x00\x00\x00'


In [16]:
#修改buf内容（这是可变buf）
p.value = b"Hi"
#输出10 'Hi\x00lo\x00\x00\x00\x00\x00'
print(sizeof(p), repr(p.raw))

10 b'Hi\x00lo\x00\x00\x00\x00\x00'


In [17]:
#创建一个unicode使用的buf，且初始化为0
pw = create_unicode_buffer(3)

#注意，这里将输出6。
print(sizeof(pw))

6


**想要创建包含 unicode 字符(对应 C 类型 wchar_t)的可变内存块，使用 create_unicode_buffer() 函数。**

---

# 类型转换

通常情况下，ctypes 会做严格的类型检查。这意味着，如果形参有一个 POINTER(c_int)指针指向一个函数或者结构体的成员域类型，那么实参只能接受相同类型的实例。
但这个规则也有例外。比如，你可以传递兼容的数组类型来代替指针类型。**例如对于 POINTER(c_int)指针类型来说，可以使用 c_int 数组来代替。**

In [18]:
class Bar(Structure):
    _fields_ = [("count", c_int), ("values", POINTER(c_int))]


bar = Bar()
bar.values = (c_int * 3)(1, 2, 3)  #数组和指针的转化
bar.count = 3
for i in range(bar.count):
    print(bar.values[i])

bar.values = None  #设置指针为NULL

1
2
3


如果一个函数参数显式声明为某种指针类型（例如 POINT(c_int) 类型），则传递该指针指向的对象类型也是可以的（例如这里可以传递 c_int），ctypes 会**自动加上 byref()函数**进行类型转换。

在 C 语言中，你可以通过强制类型转换的方法来转换不兼容的类型。ctypes 也提供了一个**转换函数 cast()**让你可以使用相同的方式进行类型转换。
cast()函数可以将一个 ctypes 指针（或数组）的实例转换成另外一个不同的指针类型（或数组）。cast()函数需要两个参数，第一个是转换前的**指针实例**，
第二个是目标**指针类型**。它返回第二个参数类型的实例，并且这个实例与第一个参数共用同一块内存。

上面定义的 Bar 结构体中，它的 value 域可以支持 POINTER(c_int)指针或者 c_int 数组，但不支持其他类型，如果需要其它类型，则可使用类型转换。

In [19]:
#报错，类型不对。需要 int * 或 int 数组。这里是byte数组。
bar.values = (c_byte * 4)()

TypeError: incompatible types, c_byte_Array_4 instance instead of LP_c_long instance

In [20]:
#正确。强制转换,把byte数组转化为int*
bar.values = cast((c_byte * 4)(), POINTER(c_int))

print(bar.values[0])

0


In [21]:
CFUNCTYPE

<function ctypes.CFUNCTYPE(restype, *argtypes, **kw)>

# 回调函数

types 允许从 python 回调中创建 c 回调函数指针。这个常常被称为回调函数。

首先，你必须为回调函数创建一个类，这个类知道**调用协议，函数返回值类型，函数接受的参数个数及类型。**

CFUNCTYPE 工厂函数使用普通 cdecl 调用协议来为回调函数创建类型。并且，在 Windows 平台，WINFUNCTYPE 工厂函数使用 stdcall 调用协议来为回调函数创建类型。
这两个工厂函数在调用时，参数表都是使用返回值作为第一个参数，而将回调函数所需要的参数作为剩下的参数。

在这里我将使用一个 c 标准库里的快排函数作为演示例子，快排是一个借助回调函数进行排序的函数。快排将会用到下面的整型数组：

```python

CMPFUNC = CFUNCTYPE(c_int, POINTER(c_int), POINTER(c_int)) #定义回调函数类型

def py_cmp_func(a, b): #实现回调函数功能
      print("py_cmp_func", a[0], b[0])
      return a[0] - b[0]

cmp_func = CMPFUNC(py_cmp_func)  #回调函数对象

IntArray5 = c_int*5
ia = IntArray5(5, 1, 7, 33, 99)
qsort = libc.qsort      #排序函数地址
qsort.restype = None    #排序函数返回值

qsort(ia, len(ia), sizeof(c_int), CMPFUNC(py_cmp_func)) #调用排序函数,传入一个回调函数对象.

for i in ia:
  print(i, end=" ")

#输出结果为： 1 5 7 33 99
```


**回调函数的重要提示：**

确保你在 C 代码的使用生命周期里保持引用 CFUNCTYPE 对象。ctypes 并不会帮你做这样的事情，如果你没有做保证，它们就会被垃圾回收，然后当你调用这个回调函数时将会导致程序崩溃。

# windows 类型

Windows API 有一些特殊之处，Windows API 函数不使用标准 C 的调用约定（前面已经提到过）。

因此需注意两点：

- LoadLibrary 时不能够使用 cdll.LoadLibrary 而使用 windll.LoadLibrary。
- 在声明回调函数指针类型的时候，不能用 CFUNCTYPE 而是用 WINFUNCTYPE。

Windows API 有很多内建类型，ctypes 内部都已经定义好了，在子模块 wintypes 下，可以直接使用。

代码举例如下：

In [69]:
from ctypes import *
from ctypes import wintypes

# HWND 等类型已定义好了，可直接使用.
WNDENUMPROC = WINFUNCTYPE(wintypes.BOOL,  #定义回调函数类型
                          wintypes.HWND, 
                          wintypes.LPARAM)

def EnumWindowsProc(hwnd, lParam):#实现回调函数功能
    length = user32.GetWindowTextLengthW(hwnd) + 1
    buffer = create_unicode_buffer(length)
    user32.GetWindowTextW(hwnd, buffer, length)
    print(buffer.value)
    return True

user32 = windll.LoadLibrary('user32.dll')    #加载dll
user32.EnumWindows(WNDENUMPROC(EnumWindowsProc), 0)


Forcepad driver tray window

Syn Zoom Window
Syn Visual Window












CandidateWindow







电池指示器

Network Flyout

将下载链接拖至窗口即可开始下载




微信
Tooltip
CandidateWindow
Mode Indicator
Mode Indicator
Mode Indicator
CandidateWindow
Mode Indicator
Mode Indicator
Mode Indicator







CandidateWindow
Mode Indicator
Mode Indicator
Mode Indicator

ctypes 使用 - Google Chrome
在python中检测按键？ - 码客 - Google Chrome
tb-gateway [C:\Users\Administrator\Desktop\git_clone\tb-gateway] - ...\total_logic.py [tb-gateway] - PyCharm
任务管理器

notebook
IDM agent for click monitoring in IE-based browsers



Internet Download Manager 6.32
DDE Server Window
smart_robot [C:\Users\Administrator\Desktop\git_clone\smart_robot] - ...\config\keyboard.py [smart_robot] - PyCharm
TIM

STATIC


FingerDrive.py - Untitled (Workspace) - Visual Studio Code





ChatContactMenu
微信
 

CandidateWindow












Mode Indicator
CiceroUIWndFrame
桌面歌词


Window
Window

Mode Indicator
Mode Indicator
Mode Indicator
Mode Indicator
Mode Ind

1

# 结构体指针

返回结构体指针的函数. c 代码如下:

```c
include <stdlib.h>
typedef struct{
  int a;
  int b;
}mystruct;

mystruct * create(){
  mystruct * s = (mystruct *)calloc(1, sizeof(mystruct));
  s->a = 100;
  s->b = 200;
  return s;
}

void destroy(mystruct * s){
  free(s);
}
```

python 代码如下：

```python
from ctypes import *  
class mystruct(Structure):
    _fields_ = [('a', c_int),('b', c_int)]

dll = cdll.LoadLibrary(dllpath) 
dll.create.restype = POINTER(mystruct) #设置返回值类型为结构体指针
p = dll.create() #调用函数
print(p.contents.a,p.contents.b) #输出 100 200
```

如果返回的是结构体数组指针，同样也是将函数返回类型设置为结构体指针，在接收到值之后，使用时加上下标即可,注意是结构体数组指针，不是结构体指针数组

python 代码如下：

```python
from ctypes import *  
class mystruct(Structure):
    _fields_ = [('a', c_int),('b', c_int)]

dll = cdll.LoadLibrary(dllpath) 
dll.create.restype = POINTER(mystruct) #设置返回值类型为结构体指针
p = dll.create() #调用函数
print(p[i].a,p[i].b) #注意是结构体数组指针，不是结构体指针数组，不然应该还需要
```

**如果是向函数内部传入一个结构体数组指针，而函数需要改变此结构体的内存，则可以作如下处理：**

返回结构体指针参数的函数. c 代码如下:

```c
include <stdlib.h>
typedef struct{
  int a;
  int b;
}mystruct;

 int change(mystruct*stru,int*num){

     for(i＝０;i < 3;i++)
      {
       stru[i].a = i;
       stru[i].b = i+1;
      }

      int n_num = 2;
      num = &n_num;
      return 0;
}
```

python 代码如下：

```python
from ctypes import *  
class mystruct(Structure):
    _fields_ = [('a', c_int),('b', c_int)]
dll = cdll.LoadLibrary(dllpath) 
stru_info= create_string_buffer(sizeof(mystruct) * NUM)
p_rec = POINTER(mystruct)(stru_info)
info_num = c_int()
ret = dll.create(p_rec, byref(info_num)) #调用函数
print(p_rec[i],p_rec[i].b)
```

**同理，如果创建int类型的数组内存，则可以用**

```python
int_buffer = create_string_buffer(sizeof(c_int) * NUM)
p_int_buffer = POINTER(c_int)(int_buffer)
```

传递参数时直接使用p_int_buffer使用时直接p_int_buffer[i]取值即可

**如果是向函数内部传入一个结构体数组指针，而函数不需要改变此结构体的内存，则可以作如下处理：**

c代码如下:

```c
include <stdlib.h>
typedef struct{
  int a;
  int b;
}mystruct;

 int change(mystruct*stru,int num){

 for(i＝０;i < num;i++)
  {
   cout<<stru[i].a<<endl;
   cout<<stru[i].b<<endl;
  }
  
  return 0;
}
```

python 代码如下：

```python
from ctypes import *  
class mystruct(Structure):
    _fields_ = [('a', c_int),('b', c_int)]
dll = cdll.LoadLibrary(dllpath) 
info_num = 3
stru_info_list = []#添加一些结构体，此处略去
stru_info_p = (mystruct*info_num)(*stru_info_list)
num = c_int(info_num)
ret = dll.create(byref(strstru_info_p), num) #调用函数
print(p_rec[i],p_rec[i].b)

同理，如果只是传递一个数组指针，不需要改变内容的话，假设传第一个int数组：
int_list = []
num = len(int_list)
p_int_list = (c_int*num)(*int_list)
```

# 解析
上述例子中，如果创建int类型的数组内存
假设c函数是int fun（int*buffer_list）#传递的是int数组的指针

int_buffer = create_string_buffer(sizeof(c_int) * NUM)
如果直接传递的参数为byref（int_buffer）
即：ret = dll.fun（byref（int_buffer））

那么对于int_buffer的使用就需要解析
首先需要取int_buffer得二进制内容，即int_buffer.raw，然后解析
buffer = int_buffer.raw
buffer_int = (struct.unpack('i', buffer[i * 4 : (i + 1) * 4]))[0]
